In [1]:
import numpy as np
import optuna
import torch

from image_classification.models.mlp import MLPClassifier
from image_classification.models.resnet import ResNetClassifier
from layers import ResNetBlock
import lightning.pytorch as pl
import lightning.pytorch.loggers as pl_loggers
import lightning.pytorch.callbacks as pl_callbacks
from image_classification.models.classification_model import ClassificationModel

from util import set_seed
from weight_init import init_for_relu
from sklearn.model_selection import ParameterGrid
from datasets import FashionMNIST
from experiment import train
import sklearn.metrics as skm
import torch

C:\Users\George Tzoupis\anaconda3\envs\ailab\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def show_classification_results(test_dl, model, classnames):
    ypred, ytrue = [], []

    model.eval()
    with torch.inference_mode():
        for x, y in test_dl:
            pred = model(x.to(model.device))
            ypred.extend(pred.argmax(-1).cpu().numpy().flatten())
            ytrue.extend(y.cpu().numpy().flatten())

    print(skm.classification_report(ytrue, ypred, target_names=classnames))
    print(skm.confusion_matrix(ytrue, ypred))

# Param Grid using sklearn

In [3]:
torch.set_float32_matmul_precision("medium")

In [5]:
pl.seed_everything(42, True)

train_dl, valid_dl, test_dl = FashionMNIST.get_dataloaders(batch_size=16,
                                                           pin_memory=True,
                                                           num_workers=4,
                                                           persistent_workers=True)
n_classes = len(train_dl.dataset.classes)

# model = ResNetClassifier(
#     n_classes=n_classes,
#     opt='AdamW',
#     lr=1e-1,
#     wd=1e-4, ).apply(init_for_relu)

param_grid = {
    'n_features': [
        # (4, 8, 16, 32, 64, 128),
        (8, 16, 32, 64, 128, 256),
        (16, 32, 64, 128, 256, 512),
    ],
    'lr': [1e-3],
    'wd': [1e-5],
}

for params in ParameterGrid(param_grid):
    # Select classification model:
    # classifier = MLPClassifier(
    #     input_sz=28 * 28,
    #     n_classes=n_classes,
    #     n_features=(16, 32,),
    # ).apply(init_for_relu)
    # or ...
    classifier = ResNetClassifier(
        n_classes=n_classes,
        n_features=params["n_features"]
    ).apply(init_for_relu)

    model = ClassificationModel(
        model=classifier,
        n_classes=n_classes,
        opt="AdamW",
        lr=params["lr"],
        wd=params["wd"]
    )

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )

    experiment_name = f"ResNet-n_layers-{len(params['n_features'])}"
    # experiment_name = f"Testing / Refactor"
    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name=experiment_name,
                                             sub_dir=model.classifier.__class__.__name__)
    trainer = pl.Trainer(max_epochs=10,
                         # limit_train_batches=10,
                         # limit_val_batches=10,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )
    trainer.fit(model=model,
                train_dataloaders=train_dl,
                val_dataloaders=valid_dl,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )

    show_classification_results(test_dl, model, test_dl.dataset.classes)


Global seed set to 42
C:\Users\George Tzoupis\anaconda3\envs\ailab\lib\site-packages\lightning\pytorch\utilities\parsing.py:197: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type             | Params
------------------------------------------------
0 | classifier | ResNetClassifier | 1.2 M 
------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.920     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


              precision    recall  f1-score   support

 T-shirt/top       0.81      0.77      0.79      1000
     Trouser       0.99      0.96      0.97      1000
    Pullover       0.82      0.68      0.75      1000
       Dress       0.75      0.90      0.82      1000
        Coat       0.66      0.84      0.74      1000
      Sandal       0.97      0.91      0.94      1000
       Shirt       0.62      0.49      0.55      1000
     Sneaker       0.89      0.92      0.91      1000
         Bag       0.97      0.95      0.96      1000
  Ankle boot       0.92      0.96      0.94      1000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000

[[765   2  15 115  10   1  82   1   9   0]
 [  1 959   3  30   5   0   0   0   2   0]
 [ 14   2 680  12 185   0 107   0   0   0]
 [ 19   4   3 900  38   0  36   0   0   0]
 [  1   1  48  57 840   0  52   0   1   0]
 [  1   0   0   0   0 911 

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    pl.seed_everything(42, True)

    params = {
        "dropout": trial.suggest_float("dropout", 0.1, 0.5, log=True),
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "wd": trial.suggest_float("wd", 1e-5, 1e-1, log=True),
        "opt": trial.suggest_categorical("opt", ["SGD", "RAdam"]),
    }

    data_module = FashionMNIST.FashionMNISTDataModule()
    n_classes = data_module.n_classes
    model = ResNetClassifier(
        n_classes=n_classes,
        **params).apply(init_for_relu)

    trainer = train(model,
                    data_module,
                    monitored_metric='accuracy/val',
                    mode='max',
                    max_epochs=10,
                    limit_train_batches=200)

    return trainer.callback_metrics["accuracy/val"].item()


pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=100, timeout=60 * 60, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
data_module = FashionMNIST.FashionMNISTDataModule()
ckpt_path = "results/FashionMNIST/version_2/checkpoints/last.ckpt"
model = ResNetClassifier.load_from_checkpoint(ckpt_path)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=1,
    monitor="accuracy/val",
    mode="max",
    filename="best-{epoch:02d}",
    save_last=True,
)

tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                         name=data_module.dataset_name,
                                         sub_dir=model.__class__.__name__)

trainer = pl.Trainer(max_epochs=40,
                     callbacks=[
                         checkpoint_callback,
                     ],
                     logger=tb_logger,
                     )

trainer.fit(model,
            data_module,
            ckpt_path=ckpt_path,
            )